# Génération automatique des liens JupyterHub UTLN, NBViewer Github et Binder

<b>ADAPTER LE TABLEAU CI-DESSOUS</b> qu décrit les notebooks.

In [448]:
notebooks=[
    # Les profils disponibles
    # pythonUTLN, datascience, javaUTLN, dbUTLN
    {"title":"Python - JHub Intro"  ,"profile":"pythonUTLN", "repo_url":"https://github.com/ebpro/JupyterHubUTLNIntro" , "branch":"develop", "url_path":"JupyterHub UTLN.ipynb"},
    # Il est possible d'avoir plusieurs Notebooks dans le même repository
    {"title":"Java - Installation"  ,"profile":"javaUTLN"   , "repo_url":"https://github.com/ebpro/javanotebook-base"   , "branch":"develop", "url_path":"01.01_InstallationCompilation.ipynb"},
    {"title":"Java - Maven"         ,"profile":"javaUTLN"   , "repo_url":"https://github.com/ebpro/javanotebook-base"   , "branch":"develop", "url_path":"01.02_MavenIntroduction.ipynb"},
    {"title":"Java - Après Java 8"  ,"profile":"javaUTLN"   , "repo_url":"https://github.com/ebpro/javanotebook-base"   , "branch":"develop", "url_path":"01.07_After Java 8.ipynb"},    
    
    {"title":"Java - JDBC"          ,"profile":"javaUTLN"   , "repo_url":"https://github.com/ebpro/javanotebook-jdbc"   , "branch":"develop", "url_path":"IntroductionJDBC.ipynb"},
    {"title":"Java - REST/JAXRS"    ,"profile":"javaUTLN"   , "repo_url":"https://github.com/ebpro/javanotebook-jaxrs"  , "branch":"develop", "url_path":"jaxrs.ipynb"},
    
    {"title":"Database - Sandox"    ,"profile":"dbUTLN"     , "repo_url":"https://github.com/ebpro/SQLNotebookDemo"     , "branch":"develop", "url_path":"SQL sandbox.ipynb "}
    ]

In [447]:
import os
from ipywidgets import interact
from urllib.parse import urlunparse, urlparse, urlencode, parse_qs, parse_qsl, quote
from IPython.display import Markdown

jupyterEnvs={'pythonUTLN'  :{'url':"https://github.com/ebpro/jupyterutln-default"        , 'branch':'develop'},
           # 'datascience" :'                                                    , 'branch':'develop'},
             'javaUTLN'    :{'url':"https://github.com/ebpro/JupyterJava", 'branch':'develop'},
             'dbUTLN'      :{'url':"https://github.com/ebpro/JupyterDB"          , 'branch':'develop'}
        }

#@interact
def make_launch_link(repo_url, profile='pythonUTLN',branch='develop', url_path=''):
    """
    Generate a launch request which clones and merges source files from a git-based
    repository.

    Args:
      profile (str): the Jupyter Hub Profile.
      branch (str): git repo branch
      hub_url (str): full hub url which needs to include scheme (http or https) and netloc (full domain).
      repo_url (str): full git repo url which needs to include scheme (http or https), netloc (full domain) and path.
      url_path (str): a path to redirect users to after the workspace has successfully spawned (started).

    Returns:
      The URLs
    """

    hub_url='https://jovem.univ-tln.fr'
    
    ghaccount,ghrepo = urlparse(repo_url).path.split('/')[-2:]
    
    gitpull_params = {"repo":repo_url,"urlpath":"lab/tree/work/assignments/"+ghrepo+"/"+url_path, "branch":branch}    
    encoded_gitpull_params = urlencode(gitpull_params)
    
    gitpull_url=f'{hub_url}/hub/user-redirect/git-pull?{encoded_gitpull_params}'

    with_profile=f'{hub_url}/hub/spawn?{urlencode({"profile":profile,"next":gitpull_url})}'

    binder_gitpull_params = {"repo":repo_url,"urlpath":"lab/tree/"+ghrepo+"/"+url_path, "branch":branch}    
    binder_ghaccount,binder_ghrepo = urlparse(jupyterEnvs[profile]["url"]).path.split('/')[-2:]
    binder_branch=jupyterEnvs[profile]["branch"]
    binder_url=f'https://mybinder.org/v2/gh/{binder_ghaccount}/{binder_ghrepo}/{binder_branch}?urlpath=git-pull%3F{quote(urlencode(binder_gitpull_params))}'

    return {'utln':with_profile,'binder':binder_url,'static':repo_url+'/blob/'+branch+'/'+url_path}    

In [449]:
make_launch_link(notebooks[0]['repo_url'],notebooks[0]['profile'], notebooks[0]['branch'],notebooks[0]['url_path'])

{'utln': 'https://jovem.univ-tln.fr/hub/spawn?profile=pythonUTLN&next=https%3A%2F%2Fjovem.univ-tln.fr%2Fhub%2Fuser-redirect%2Fgit-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Febpro%252FJupyterHubUTLNIntro%26urlpath%3Dlab%252Ftree%252Fwork%252Fassignments%252FJupyterHubUTLNIntro%252FJupyterHub%2BUTLN.ipynb%26branch%3Ddevelop',
 'binder': 'https://mybinder.org/v2/gh/ebpro/jupyterutln-default/develop?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Febpro%252FJupyterHubUTLNIntro%26urlpath%3Dlab%252Ftree%252FJupyterHubUTLNIntro%252FJupyterHub%2BUTLN.ipynb%26branch%3Ddevelop',
 'static': 'https://github.com/ebpro/JupyterHubUTLNIntro/blob/develop/JupyterHub UTLN.ipynb'}

In [453]:
%load_ext jinja_yaml_magic
%reload_ext jinja_yaml_magic

The jinja_yaml_magic extension is already loaded. To reload it, use:
  %reload_ext jinja_yaml_magic


Les liens sont présentés via un template Jinja :

In [456]:
%%jinja_template MyNotebooks
<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous">
<table class="table table-hover">
 <thead>
  <tr>
    <th scope="col">Notebook</th>
    <th scope="col">Statique</th>
    <th scope="col" colspan="2">Dynamique</th>
  </tr> 
  <tr>
    <th scope="col"></th>
    <th scope="col"></th>
    <th scope="col">Binder</th>
    <th scope="col">UTLN</th>
  </tr> 
 </thead>
 <tbody>
  {% for nb in notebooks %}
  {% set links=make_launch_link(nb.repo_url,nb.profile, nb.branch,nb.url_path) %}
  <tr>
    <th scope="row">{{ nb["title"] }}</th>
    <td scope="row"><a href="{{ links.static }}"><img  width="100" alt="nbviewer" src="https://user-images.githubusercontent.com/2791223/29387450-e5654c72-8294-11e7-95e4-090419520edb.png"/></a>
    <td scope="row"><a href="{{ links.binder }}"><img  width="64" alt="Binder" src="https://binderhub.readthedocs.io/en/latest/_static/logo.png"/></a>
    <td scope="row"><a href="{{ links.utln }}"><img  width="64" alt="Université de Toulon - UTLN" src="https://www.univ-tln.fr/squelettes/images/logos/logo-utln.png"/></a>
  </tr>
  {%- endfor %}
 </tbody>
<table>

## Le rendu

In [457]:
%jinja --template MyNotebooks --html

## Le source HTML

In [458]:
%jinja --template MyNotebooks

<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous">
<table class="table table-hover">
 <thead>
  <tr>
    <th scope="col">Notebook</th>
    <th scope="col">Statique</th>
    <th scope="col" colspan="2">Dynamique</th>
  </tr> 
  <tr>
    <th scope="col"></th>
    <th scope="col"></th>
    <th scope="col">Binder</th>
    <th scope="col">UTLN</th>
  </tr> 
 </thead>
 <tbody>
  
  
  <tr>
    <th scope="row">Python - JHub Intro</th>
    <td scope="row"><a href="https://github.com/ebpro/JupyterHubUTLNIntro/blob/develop/JupyterHub UTLN.ipynb"><img  width="100" alt="nbviewer" src="https://user-images.githubusercontent.com/2791223/29387450-e5654c72-8294-11e7-95e4-090419520edb.png"/></a>
    <td scope="row"><a href="https://mybinder.org/v2/gh/ebpro/jupyterutln-default/develop?urlpath=git-pull%3Frepo%3Dhttps%253A%252F%252Fgithub.com%252Feb